In [ ]:
import pathlib
import Quartz
import Vision
from Cocoa import NSURL
from Foundation import NSDictionary
from wurlitzer import pipes
import pandas as pd
from PIL import Image
import os

def image_to_text(img_path, lang="eng"):
    """Extract text from an image using Apple's Vision framework."""
    input_url = NSURL.fileURLWithPath_(img_path)

    with pipes() as (out, err):
        # Capture stdout and stderr from system calls
        input_image = Quartz.CIImage.imageWithContentsOfURL_(input_url)

    vision_options = NSDictionary.dictionaryWithDictionary_({})
    vision_handler = Vision.VNImageRequestHandler.alloc().initWithCIImage_options_(
        input_image, vision_options
    )
    results = []
    handler = make_request_handler(results)
    vision_request = Vision.VNRecognizeTextRequest.alloc().initWithCompletionHandler_(handler)
    error = vision_handler.performRequests_error_([vision_request], None)

    # Join the results into a single string
    text_result = ' '.join([text for text, _ in results])
    return text_result

def make_request_handler(results):
    """Create a request handler to process Vision text recognition results."""
    if not isinstance(results, list):
        raise ValueError("results must be a list")

    def handler(request, error):
        if error:
            print(f"Error! {error}")
        else:
            observations = request.results()
            for text_observation in observations:
                recognized_text = text_observation.topCandidates_(1)[0]
                results.append([recognized_text.string(), recognized_text.confidence()])
    return handler

def process_images_in_folder(folder_path):
    """Process all images in the specified folder and save results to a DataFrame."""
    folder_path = pathlib.Path(folder_path)
    if not folder_path.is_dir():
        print("Invalid folder path")
        return

    data = []
    for img_file in folder_path.glob('*.*'):
        if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
            try:
                text = image_to_text(str(img_file.resolve()))
                data.append({'filename': img_file.name, 'text': text})
            except Exception as e:
                print(f"Failed to process {img_file}: {e}")

    # Create a DataFrame and save to CSV
    df = pd.DataFrame(data)
    df.to_csv('image_text_results.csv', index=False)
    print("Results saved to image_text_results.csv")

def main():
    # Prompt user for folder path
    
    process_images_in_folder('./downloaded_images')

if __name__ == "__main__":
    main()
